In [1]:
from paddleocr import PaddleOCR, draw_ocr

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
ocr = PaddleOCR(use_angle_cls=False, lang="ch", use_gpu=True,)  # need to run only once to download and load model into memory

[2024/09/09 18:20:40] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_b

In [2]:
from PIL import Image

In [3]:
font_path = './doc/fonts/simfang.ttf'

In [4]:
from glob import glob
from pathlib import  Path
import os
img_folder = Path(r'C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area')
img_files = glob(os.path.join(img_folder / '**/*.jp*'), recursive=True)
print(img_files)

['C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_105946.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_132525.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_140224.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_140915.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_152937.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_153357.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_173021.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_204937.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_232901.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240816_001716.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240816_072949.jpg', 'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-

In [21]:
img_path = img_files[3]
result = ocr.ocr(img_path, cls=False)
# for idx in range(len(result)):
#     res = result[idx]
#     for line in res:
#         print(line)

# 显示结果

result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
im_show = draw_ocr(image, boxes, txts, scores, font_path=font_path)
im_show = Image.fromarray(im_show)
im_show.save('result.jpg')

[2024/09/09 17:52:06] ppocr DEBUG: dt_boxes num : 143, elapsed : 4.674571514129639
[2024/09/09 17:52:07] ppocr DEBUG: rec_res num  : 143, elapsed : 0.654738187789917


In [40]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr目前支持的多语言语种可以通过修改lang参数进行切换
# 例如`ch`, `en`, `fr`, `german`, `korean`, `japan`
PAGE_NUM = 10 # 将识别页码前置作为全局，防止后续打开pdf的参数和前文识别参数不一致 / Set the recognition page number
pdf_path = r'C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L6A\Aug 15-L6A-Part 1.pdf'
ocr = PaddleOCR(use_angle_cls=True, lang="ch", page_num=PAGE_NUM)  # need to run only once to download and load model into memory
# ocr = PaddleOCR(use_angle_cls=True, lang="ch", page_num=PAGE_NUM,use_gpu=0) # 如果需要使用GPU，请取消此行的注释 并注释上一行 / To Use GPU,uncomment this line and comment the above one.
result = ocr.ocr(pdf_path, cls=True)
for idx in range(len(result)):
    res = result[idx]
    if res == None: # 识别到空页就跳过，防止程序报错 / Skip when empty result detected to avoid TypeError:NoneType
        print(f"[DEBUG] Empty page {idx+1} detected, skip it.")
        continue
    for line in res:
        print(line)
# 显示结果
import pymupdf
from PIL import Image
import cv2
import numpy as np
imgs = []
with pymupdf.open(pdf_path) as pdf:
    for pg in range(0, PAGE_NUM):
        page = pdf[pg]
        mat = pymupdf.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=pymupdf.Matrix(1, 1), alpha=False)
        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)
        
for idx in range(len(result)):
    res = result[idx]
    if res == None:
        continue
    image = imgs[idx]
    boxes = [line[0] for line in res]
    txts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]
    im_show = draw_ocr(image, boxes, txts, scores, font_path='doc/fonts/simfang.ttf')
    im_show = Image.fromarray(im_show)
    im_show.save('result_page_{}.jpg'.format(idx))

[2024/09/09 16:43:02] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=10, det_algorithm='DB', det_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec

In [ ]:
python predict_system.py ^
    --image_dir="C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L6A\Aug 15-L6A-Part 1.pdf" ^
    --recovery=True ^
    --use_pdf2docx_api=True ^
    --output=../output/

In [ ]:
# 安装 paddleocr，推荐使用2.6版本
pip3 install "paddleocr>=2.6"
paddleocr --image_dir=ppstructure/docs/recovery/UnrealText.pdf --type=structure --recovery=true --use_pdf2docx_api=true --output=../output/

In [ ]:
# 中文测试图
paddleocr --image_dir="C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L5C\IMG_5616.jpeg" --type=structure --recovery=true
# 英文测试图
paddleocr --image_dir=ppstructure/docs/table/1.png --type=structure --recovery=true --lang='en'
# pdf测试文件
paddleocr --image_dir=ppstructure/docs/recovery/UnrealText.pdf --type=structure --recovery=true --lang='en'

In [ ]:
python predict_system.py --det_model_dir=../inference/ch_PP-OCRv4_det_infer ^
                          --rec_model_dir=../inference/ch_PP-OCRv4_rec_infer ^
                          --table_model_dir=../inference/ch_ppstructure_mobile_v2.0_SLANet_infer ^
                          --layout_model_dir=../inference/picodet_lcnet_x1_0_fgd_layout_cdla_infer ^
                          --image_dir=./docs/table/1.png ^
                          --rec_char_dict_path=../ppocr/utils/ppocr_keys_v1.txt ^
                          --table_char_dict_path=../ppocr/utils/dict/table_structure_dict_ch.txt ^
                          --output=../output ^
                          --vis_font_path=../doc/fonts/simfang.ttf

In [ ]:
python predict_system.py --det_model_dir=../inference/ch_PP-OCRv4_det_infer 
                          --rec_model_dir=../inference/ch_PP-OCRv4_rec_infer 
                          --table_model_dir=../inference/ch_ppstructure_mobile_v2.0_SLANet_infer 
                          --layout_model_dir=../inference/picodet_lcnet_x1_0_fgd_layout_cdla_infer 
                          --image_dir=./docs/table/1.png 
                          --rec_char_dict_path=../ppocr/utils/ppocr_keys_v1.txt 
                          --table_char_dict_path=../ppocr/utils/dict/table_structure_dict_ch.txt 
                          --output=../output 
                          --vis_font_path=../doc/fonts/simfang.ttf

In [ ]:
python paddleocr.py --image_dir=ppstructure/docs/table/1.png --type=structure --output=./output

In [68]:
import os
import cv2
import openpyxl
from openpyxl.cell import cell
from tablepyxl.style import TableCell
from paddleocr import PPStructure, draw_structure_result, save_structure_res

In [70]:
table_engine = PPStructure(show_log=True, use_gpu=False, image_orientation=False, output='./output/', use_onnx=False, layout_model_dir=r"./inference/picodet_lcnet_x1_0_fgd_layout_cdla_infer")


[2024/09/09 19:05:45] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Tristan.Lo/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_

In [9]:
os.path.join(r'C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area', r'L1-到港層\20240815_132525.jpg')

'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_132525.jpg'

In [ ]:
cv2.imread(r"C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L11\IMG_5457.jpeg")

In [50]:
print(rf'{img_files[3]}')

C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L1-到港層\20240815_140915.jpg


In [71]:
import cv2

# Load an image from file
# image_path = img_files[0]
# image = cv2.imread(image_path,0)
image = cv2.imread(r"C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L11\IMG_5457.jpeg")

# Check if the image was successfully loaded
if image is not None:
    print('Image loaded successfully!')
    # Display the image in a window
    # cv2.imshow('Loaded Image', image)
    # Wait for a key press and close the window
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
else:
    print('Failed to load image.')


Image loaded successfully!


In [ ]:
# import os
# import cv2
# from paddleocr import PPStructure,draw_structure_result,save_structure_res

# table_engine = PPStructure(show_log=True)

# save_folder = './output'
# img_path = 'ppstructure/docs/table/1.png'
# img = cv2.imread(img_path)
# result = table_engine(img)
# save_structure_res(result, save_folder,os.path.basename(img_path).split('.')[0])

# for line in result:
#     line.pop('img')
#     print(line)

# from PIL import Image

# font_path = 'doc/fonts/simfang.ttf' # PaddleOCR下提供字体包
# image = Image.open(img_path).convert('RGB')
# im_show = draw_structure_result(image, result,font_path=font_path)
# im_show = Image.fromarray(im_show)
# im_show.save('result.jpg')

In [ ]:
img_path = './ppstructure/docs/table/1.png'
img = cv2.imread(img_path)
result = table_engine(img)
save_structure_res(result, './output', 'output.xlsx')

In [13]:
image_path = r"C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L5C\IMG_3299.jpeg" # img_files[0]
image = cv2.imread(image_path)

In [ ]:
r"C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L11\IMG_5457.jpeg"

In [86]:
img_files[3]

'C:\\Users\\Tristan.Lo\\Downloads\\Aug 15-Terminal Area\\L1-到港層\\20240815_140915.jpg'

In [77]:
# image_path = r"C:\Users\Tristan.Lo\Downloads\Aug 15-Terminal Area\L5C\IMG_3299.jpeg" # img_files[0]
# image = cv2.imread(image_path)

# Perform structure analysis
result = table_engine(img_files[3])

# Draw and save the structure result
# image_with_result = draw_structure_result(image, result, font_path=font_path)
# cv2.imwrite('output_image.jpg', image_with_result)

# Save the structure result in a structured format
# save_structure_res(result, './output', 'output.xlsx')


[2024/09/09 19:08:08] ppocr DEBUG: dt_boxes num : 143, elapsed : 0.21361565589904785
[2024/09/09 19:08:10] ppocr DEBUG: rec_res num  : 143, elapsed : 1.7632639408111572
[2024/09/09 19:08:11] ppocr DEBUG: dt_boxes num : 144, elapse : 0.21312165260314941
[2024/09/09 19:08:13] ppocr DEBUG: rec_res num  : 144, elapse : 1.7362596988677979


In [73]:
import pandas as pd

In [87]:
assert(len(result) == 1)
pd.read_html(result[0]['res']['html'])[0].to_csv('output.csv', encoding='utf_8_sig', index=False)